In [60]:
import os
import json
import pandas as pd
import pyarrow as pa
import pyarrow.parquet as pq
from datetime import datetime

Para poder tener una mirada más amplia y prolija sobre la data de cada estado, decidí unificar los json dentro de las carpetas de cada estado.

Ya que dentro de cada archivo cada linea representa un json, tomé la decisión de iterar cada linea/json de archivo y convertirlo en un dataframe. Como este proceso debe repetirse por cada archivo dentro de la carpeta estado, no instancio aún el archivo dentro de la siguiente función. Más adelante con el funcionamiento correcto de cada función voy a unificar los procesos para operar de manera prolija sobre la información.

In [61]:
def iterador_linea(archivo_json):
    #archivo_json = "../reviews-estados/review-Alabama/1.json"
    objetos_json = []

    #iteracion del archivo linea por linea, agregando nuevos objetos al array vacio.
    with open(archivo_json, "r") as file:
        for linea in file:
            objeto = json.loads(linea)
            objetos_json.append(objeto)
            
    #creo el dataframe donde se va a alojar esta nueva info.
    df = pd.DataFrame(objetos_json)
    return df

In [62]:
#prueba
df1 = "../reviews-estados/review-Alabama/1.json"
iterador_linea(df1)

,user_id,name,time,rating,text,pics,resp,gmap_id
0,114043824230907811356,Kanisha Mixon,1597168272670,5,Very Personable staff! Beautiful and clean env...,None,None,0x8862134e67ff5c87:0x38b5e2ae99cd1fcf
1,116009008420407071921,Brandie Hodges,1609899039594,5,Best clothing intown,None,None,0x8862134e67ff5c87:0x38b5e2ae99cd1fcf
2,106239928178034609776,Sharon King,1547235290843,4,None,None,None,0x8862134e67ff5c87:0x38b5e2ae99cd1fcf
3,104970056115801744726,Veronica Pierce,1517709403534,5,None,None,None,0x8862134e67ff5c87:0x38b5e2ae99cd1fcf
4,110587471700525107383,Whitney Waldon Collier,1535245718492,5,None,None,None,0x8862134e67ff5c87:0x38b5e2ae99cd1fcf
...,...,...,...,...,...,...,...,...
149995,116508195431461017782,Lee Lee,1601666489107,5,Just a good old hardware friendly people,None,None,0x889a89fd92d55353:0x78e335f5e98e75b2
149996,108020832340640374305,Hayward Salter,1621994613217,5,Good people,None,None,0x889a89fd92d55353:0x78e335f5e98e75b2
149997,105464581418690693091,James McBride,1601863228986,4,Good for locals needing supplies to fix damage...,None,None,0x889a89fd92d55353:0x78e335f5e98e75b2
149998,111886905477779410743,Karen Sullivan,1593013285677,1,Very rude female employees at the cash counter...,None,None,0x889a89fd92d55353:0x78e335f5e98e75b2


Ahora quiero lograr una función que itere archivo por archivo de la carpeta-estado que elija, aplicando la conversión de cada archivo a dataframe y luego unirlos en un unico dataframe.

In [64]:
def iterador_archivo(carpeta):
    dataframes = []
    #iteracion archivo por archivo, aplicando la función creada anteriormente para transformarlos.
    for archivo in os.listdir(carpeta):
        ruta_archivo = os.path.join(carpeta, archivo)
        if os.path.isfile(ruta_archivo):
            dataframes.append(iterador_linea(ruta_archivo))
    df = pd.concat(dataframes, axis=0, ignore_index=True)
    return df

Corroboro que este metodo esté funcionando correctamente.

In [65]:
alabama = "../reviews-estados/review-Alabama"
df = iterador_archivo(alabama)

Una vez la información del estado unificada en un solo dataframe, me voy a quedar unicamente con todos aquellos registros que daten del 2020 en adelante.
Para esto, tengo que transformar el formato de time, que se encuentra en timestamp Unix (segundos transcurridos desde el 1 de enero de 1970) a formato date YYYY-MM-DD y luego filtrar la informacion con la que me quiero quedar.

In [66]:
def filter_year(dataframe):
    dataframe['time'] = pd.to_datetime(dataframe['time'], unit='ms', utc=True)
    df_filtrado = dataframe[dataframe['time'].dt.year >= 2020]
    return df_filtrado

Corroboro que esta función me devuelva correctamente la data del 2020 en adelante

In [67]:
df = filter_year(df)

Ahora debo unificar todas estas acciones en un solo metodo.

In [68]:
date_min = df['time'].min()
date_max = df['time'].max()

#se imprimen tanto la menor fecha registrada en el dataframe como la maxima fecha en el mismo.
print("Fecha mínima:", date_min)
print("Fecha máxima:", date_max)

Fecha mínima: 2020-01-01 00:02:02.297000+00:00
Fecha máxima: 2021-09-09 01:02:47.785000+00:00


Una vez chequeada la funcionalidad de cada paso por separado, creo una función cual va a unificar todos estos procesos y va a recorrer carpeta por carpeta, transformando los archivos y guardandolos en un nuevo archivo parquet.

La siguiente función se encarga de convertir el dataframe instanciado en un archivo .parquet, se toma el nombre de la carpeta para nombrar el archivo, para poder identificar facilmente los archivos cuando suceda la conversion.

In [77]:
def conversor_parquet(dataframe, name_folder):
    name_parquet = f"{name_folder}.parquet"
    dataframe.to_parquet(name_parquet, index=False)
    print(f"DataFrame guardado en '{name_parquet}'")

In [70]:
conversor_parquet(df)

DataFrame guardado en 'reviews-jupiter.parquet'


In [78]:
def iterador_carpeta(carpeta_raiz):
    for ruta_actual, carpetas_sub, _ in os.walk(carpeta_raiz):
        for carpeta_sub in carpetas_sub:
            ruta_carpeta_sub = os.path.join(ruta_actual, carpeta_sub)
            dataframe = iterador_archivo(ruta_carpeta_sub)
            dataframe_filtrado = filter_year(dataframe)
            conversor_parquet(dataframe_filtrado, ruta_carpeta_sub)

In [80]:
carpeta_principal = "../reviews-estados/"
print(carpeta_principal)
iterador_carpeta(carpeta_principal)

../reviews-estados/
DataFrame guardado en '../reviews-estados/review-Alabama.parquet'
DataFrame guardado en '../reviews-estados/review-Alaska.parquet'
DataFrame guardado en '../reviews-estados/review-Arizona.parquet'
DataFrame guardado en '../reviews-estados/review-Arkansas.parquet'
DataFrame guardado en '../reviews-estados/review-California.parquet'
DataFrame guardado en '../reviews-estados/review-Colorado.parquet'
DataFrame guardado en '../reviews-estados/review-Connecticut.parquet'
DataFrame guardado en '../reviews-estados/review-Delaware.parquet'
DataFrame guardado en '../reviews-estados/review-District_of_Columbia.parquet'
DataFrame guardado en '../reviews-estados/review-Florida.parquet'
DataFrame guardado en '../reviews-estados/review-Georgia.parquet'
DataFrame guardado en '../reviews-estados/review-Hawaii.parquet'
DataFrame guardado en '../reviews-estados/review-Idaho.parquet'
DataFrame guardado en '../reviews-estados/review-Illinois.parquet'
DataFrame guardado en '../reviews-es